In [27]:
import pandas as pd 
df = pd.read_csv("listings.csv")
df_reviews= pd.read_csv("reviews.csv")

In [28]:
df = df[['host_id','number_of_reviews','neighbourhood_cleansed','host_acceptance_rate','host_response_rate','room_type','amenities',"price",'host_verifications','bathrooms_text','description','host_name','id']]



In [29]:
df["amenities"]=df["amenities"].apply(lambda x: x.count(",")+1)

df['price']=df['price'].str.replace('$','')
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype(float)

df['description'] = df['description'].str.len()

indexNames = df[ df['price'] == 0 ].index

df=df.drop(index=indexNames)

df["bathrooms_text"] = df["bathrooms_text"].str.lower()
df["bathrooms_text"] = df["bathrooms_text"].str.replace('shared half-bath','1 shared half-bath')
df["bathrooms_text"] = df["bathrooms_text"].str.replace('private half-bath','1 private half-bath')
df["bathrooms_text"] = df["bathrooms_text"].str.replace('half-bath','1 half-bath')

df[['number','type']] = df["bathrooms_text"].str.split(' ', 1, expand=True)
df["number"]=df["number"].astype(float)
df['listing_id'] = df['id']
df=df.dropna()


/tmp/ipykernel_28351/3350334474.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price']=df['price'].str.replace('$','')
/tmp/ipykernel_28351/3350334474.py:18: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['number','type']] = df["bathrooms_text"].str.split(' ', 1, expand=True)


In [30]:
df_reviews= df_reviews[['listing_id','reviewer_name']]

In [31]:
df.to_csv('n_listings.csv')

In [32]:
df_reviews.to_csv('n_reviews.csv')

In [ ]:
df['type'].value_counts()

bath                   3836
baths                  1502
shared bath             699
private bath            651
shared baths             87
shared 1 half-bath       13
half-bath                 6
private 1 half-bath       4
Name: type, dtype: int64

Calculez le nombre d’hosst et le nombre de review par quartier. (
neighbourhood_cleansed)?

In [ ]:
df_neighbourhood_cleansed = df[['host_id','number_of_reviews','neighbourhood_cleansed']]
df_neighbourhood_cleansed.groupby(by="neighbourhood_cleansed").count()

,host_id,number_of_reviews
neighbourhood_cleansed,,
3M-Bourgailh,8,8
Ambars-et-Lagrave,48,48
Ambs,46,46
Arago-La Chataigneraie,40,40
Arlac,56,56
...,...,...
Sardine,16,16
Talence,279,279
Toctoucau,16,16


Quels sont les pourcentage d’acceptation et de réponse moyen des hosts?

In [ ]:
df_acce_rate = df_listing["host_response_rate"].dropna().apply(lambda x : x.replace("%","")).astype("int").mean()
df_reponse_rate = df_listing["host_acceptance_rate"].dropna().apply(lambda x : x.replace("%","")).astype("int").mean()
print(df_acce_rate )
print(df_reponse_rate)

0         99%
1        100%
3         78%
4        100%
5        100%
         ... 
10877     99%
10879      0%
10881     90%
10882    100%
10883    100%
Name: host_acceptance_rate, Length: 6798, dtype: object
0        100%
1        100%
3        100%
4        100%
5        100%
         ... 
10877     99%
10879    100%
10881    100%
10882    100%
10883    100%
Name: host_response_rate, Length: 6798, dtype: object


Quel pourcentage des hosts on une vérification

In [ ]:
from ast import literal_eval

def pourcentage_moyen_vérification(df):
    serie_verification =df['host_verifications']
    serie_length =serie_verification.size
    n_email = 0
    n_telephone = 0
    n_work_email = 0

    for valitation in serie_verification:
        val = literal_eval(valitation)
        for ele in val:
            if ele == 'email':
                n_email+=1
            elif ele == 'phone':
                n_telephone+=1
            elif ele == 'work_email':
                n_work_email+=1
    pourcen_telephone = n_telephone/serie_length 
    pourcen_email = n_email/serie_length
    pourcen_work_mail = n_work_email/serie_length

    # print(pourcen_telephone,pourcen_email,pourcen_work_mail)

    print("le pourcentage de validation par n_email est {0:.2f}".format(pourcen_email))
    print("le pourcentage de validation par n_telephone est {0:.2f}".format(pourcen_telephone))
    print("le pourcentage de validation par n_work_email est {0:.2f}".format(pourcen_work_mail))

pourcentage_moyen_vérification(df)

le pourcentage de validation par n_email est 0.92
le pourcentage de validation par n_telephone est 1.00
le pourcentage de validation par n_work_email est 0.07


En fonction du type de room, calculez le nombre moyen d’amenities et l'écart type.

In [ ]:
df_room = df[["room_type","amenities"]]
df_room.groupby(by="room_type").agg(["mean","std"])


amenities           
                      mean        std
room_type                            
Entire home/apt  31.788868  13.564517
Hotel room       30.483871  15.112624
Private room     24.727332  12.802753
Shared room      19.371429   7.712044

Quel est le prix median par type de room, le maximum, le minimum, le premier et le troisième quartile?


In [ ]:
df_price =df[["room_type","price"]]
df_price.groupby('room_type').describe()

price                                                     \
                  count        mean         std   min    25%    50%    75%   
room_type                                                                    
Entire home/apt  5210.0  124.841075  109.448398  15.0   65.0   90.0  145.0   
Hotel room         31.0  131.741935   62.493182  32.0  100.0  120.0  143.0   
Private room     1522.0   61.038108   60.907275  17.0   34.0   44.0   60.0   
Shared room        35.0   59.114286   51.389957  21.0   42.0   46.0   49.5   

                         
                    max  
room_type                
Entire home/apt  2600.0  
Hotel room        286.0  
Private room     1000.0  
Shared room       275.0

Combien y-a-t-il d’annonces par "valeur de salle de bain". (0.25 / 0.5 / 1 / 1.5 ….)

In [ ]:
import numpy as np

df['type'] =df['type'].apply(lambda x : 1 if x=="bath" else ( 1 if x =='baths' else (0.5 if x=='shared bath'else( 2 if x=='private bath' else(2 if x=='shared baths' else(0.5 if x == 'shared 1 half-bath'else(0.5)) )))))

df["number"]=df["number"].astype(float)

df["points"] = df[['number','type']].apply(lambda x : x[0]*x[1], axis=1)

df["points"].value_counts()


# df["new_col"] = df[["taille","nom"]].apply(lambda x: x[0]+len(x[1]), axis=1)

1.0    3836
2.0    1455
0.5     722
1.5     316
3.0     193
2.5     114
4.0      56
0.0      31
5.0      28
3.5      24
6.0       8
4.5       4
8.0       4
5.5       3
6.5       2
7.0       1
7.5       1
Name: points, dtype: int64

In [ ]:
df[['number_of_reviews','description']]

,number_of_reviews,description
0,15,1000.0
1,3,912.0
3,377,1000.0
4,491,1000.0
5,53,1000.0
...,...,...
10877,6,1000.0
10879,11,1000.0
10881,82,1000.0
10882,44,809.0


il y a t’il une corrélation entre le nombre de reviews et la taille de la description en terme de nombre de caractères?

In [ ]:
df[['number_of_reviews','description']].corr()


,number_of_reviews,description
number_of_reviews,1.00000,0.15705
description,0.15705,1.00000


Quel est ce pourcentage général de séjour où le nom de l'host est identique au nom du reviewer

In [ ]:
df1 = df[['listing_id','host_name']]
df2 = df_reviews[['listing_id','reviewer_name']]
df3 = df1.merge(df2, how='right')
df3.dropna()
(df3['host_name'] ==df3['reviewer_name']).sum()/len(df_reviews)


0.0015211061640705585